Restarting python kernel relinquish the the serial control. Need to restart scanbox (in matlab?) to regain control over pyserial.

In [2]:
from scanbox.nlw_controller import *
from scanbox.utils import *
get_config()

{'cameras': {'one_photon': {'binning': 4,
   'cam_id': 0,
   'driver': 'basler',
   'exposure': 100}},
 'data_path': 'C:\\Users\\Burgess Lab\\2pdata',
 'motors': {'knobby_port': 'COM13',
  'knobby_version': 9,
  'motors_baudrate': 57600,
  'motors_port': 'COM5'},
 'network_cmd_server': 7001,
 'neurolabware_box': {'firmware': '4.5',
  'master_port': None,
  'slave_port': None},
 'twophoton': {'acquisition_nbuffers': 16,
  'acquisition_trigger_level': 160,
  'acquisition_trigger_slope': 'positive',
  'bidi_shift': 0,
  'bishift': [-71, -54, -14, -9, -21, -29, -38, -42, -48, -54, -59, -65, -70],
  'deadband': [0, 0],
  'dv_galvo': 64,
  'gain_galvo': [1.0,
   1.1892,
   1.4142,
   1.6818,
   2.0,
   2.3784,
   2.8284,
   3.3636,
   4.0,
   4.7568,
   5.6569,
   6.7272,
   8.0],
  'gain_resonant_multiplier': 1.0,
  'hsync_sign': 0,
  'laser_frequency': 80000000,
  'margin': 20,
  'objectives': ['Nikon 16x_0.8w_WD3.0'],
  'pmt_acquisition_range': [-1, 1],
  'resonant_frequency': 8000,
  'sy

In [1]:
from scanbox.nlw_controller import *
from scanbox.utils import *
config = get_config()

box = ScanboxController('COM4',preferences = config['twophoton'])

new code


In [9]:
box.get_version()

[Scanbox] Master version 1.14.0


(1, 14, 0)

In [8]:
box.abort()

[Scanbox] Stopped scanning.


In [ ]:
box.abort()
box.set_mirror_position(0)

In [ ]:
box.exit_flag = True
box.usb.close()

Configuring the API? Has something to do with the microscope?

In [15]:
import os
# os.chdir("../")
os.getcwd()

'c:\\Users\\Burgess Lab\\GitHub\\scanbox'

In [16]:
import atsapi as ats
board = ats.Board(systemId=1, boardId=1)
# Configure the capture clock (trigger external/laser)
board.setCaptureClock(source = ats.FAST_EXTERNAL_CLOCK, #2
                      rate = ats.SAMPLE_RATE_USER_DEF,  #0x40, 64, this is base-16 
                      edge = ats.CLOCK_EDGE_RISING,  # 0 
                      decimation = 0)                        

In [ ]:
board.getChannelInfo()

(c_ulong(1073741824), c_byte(14))

: 

In [ ]:
# ats.CHANNEL_A = 1, ats.CHANNEL_B = 2,ats.CHANNEL_C = 4,ats.CHANNEL_D = 8
channel_list = [ats.CHANNEL_A,ats.CHANNEL_B,ats.CHANNEL_C,ats.CHANNEL_D]

# configure the PMT channels
for channel in channel_list:
    board.inputControl(channel=channel,
                      coupling= ats.DC_COUPLING,  #2
                      inputRange = ats.INPUT_RANGE_PM_1_V, #10
                      impedance = ats.IMPEDANCE_50_OHM)  #2
    
# Configure the trigger
board.setExternalClockLevel(65) # level in percent for the external clock (laser)
hsync_value = config['twophoton']['hsync_sign']  #0
trigger_level = config['twophoton']['acquisition_trigger_level'] #160
board.setTriggerOperation(operation = ats.TRIG_ENGINE_OP_J,  #0
                          engine1 = ats.TRIG_ENGINE_J,  #0
                          source1 = ats.TRIG_EXTERNAL, #2
                          slope1 = ats.TRIGGER_SLOPE_POSITIVE + int(hsync_value), # this depends on the HSYNC
                          level1 = int(trigger_level), # trigger level from 0 (-range) to 255 (+range)
                          engine2 = ats.TRIG_ENGINE_K,  #1
                          source2 = ats.TRIG_DISABLE,  #3
                          slope2 = ats.TRIGGER_SLOPE_POSITIVE,  #1
                          level2 = 128)  # where did this 128 come from?

board.setExternalTrigger(coupling = ats.DC_COUPLING,  #2
                        range = ats.ETR_TTL)  #2
board.setTriggerDelay(delay_samples=0) # no delay
board.setTriggerTimeOut(timeout_clocks = 0) # wait forever (0)
board.abortAsyncRead()

scanmode = config['twophoton']['unidirectional'] #'unidirectional'
laser_frequency = config['twophoton']['laser_frequency'] #80000000
resonant_frequency = config['twophoton']['resonant_frequency'] #8000
samples_per_pixel = int(np.round(laser_frequency / resonant_frequency*4 / 10000))

# configure the buffers
max_samples_per_record,bits_per_sample = board.getChannelInfo()
nlines = 512

if scanmode in [0,'uni','unidirectional', True]:  # I added a "True" statement to match the config file
    post_trigger_samples = 5000*samples_per_pixel/4 # just one line...
    samples_per_record =  post_trigger_samples      # 10000/4 (1 sample every laser clock) samples per scan (back and forth)
    records_per_buffer = int(nlines)
else: # bidirectional
    post_trigger_samples = 9000*samples_per_pixel/4
    samples_per_record =  post_trigger_samples
    records_per_buffer = int(nlines/2)

# TODO: define the pixel_lookuptables here

# allocate the buffers
bytes_per_sample = 2;
nchannels = 4
nframes = 10 # continuous?

samples_per_buffer = samples_per_record * records_per_buffer * nchannels
bytes_per_buffer   = samples_per_buffer * bytes_per_sample

# allocate DMA buffer
nbuffers = 16
buffers = [ats.DMABuffer(c_sample_type = ctypes.c_uint16,
                         size_bytes = int(bytes_per_buffer)) for b in range(nbuffers)]

board.setRecordSize(preTriggerSamples = int(0),
                    postTriggerSamples = int(post_trigger_samples))


channel_selection = [1,1,1,1]

# configure acquisition
chan_sel = 0
for sel,channel in  zip(channel_selection, channel_list):
    if sel:
        chan_sel |=  channel        

board.beforeAsyncRead(
    channels = chan_sel,
    transferOffset = 0,
    samplesPerRecord = int(samples_per_record),
    recordsPerBuffer = int(records_per_buffer),
    recordsPerAcquisition = int(records_per_buffer*nframes),
    flags = ats.ADMA_EXTERNAL_STARTCAPTURE | ats.ADMA_NPT | ats.ADMA_INTERLEAVE_SAMPLES)

# post the buffers
for b in buffers:
    board.postAsyncBuffer(buffer=b.addr, bufferLength= int(bytes_per_buffer))

board.startCapture()
buffers_completed = 0
buffer_index = np.mod(buffers_completed, nbuffers)
# process a buffer and submit to the queue
b = buffers[buffers_completed % nbuffers]
try:
    board.waitAsyncBufferComplete(buffer=b.addr, timeout_ms = int(5000))
except Exception as err:
    if 'Timeout' in err.args:
        print('Timed out')
    else:
        raise(err)
print(b.buffer)

NameError: name 'board' is not defined

In [ ]:
samples_per_pixel

In [ ]:
nchannels = np.sum(channel_selection)

frame = np.array(b.buffer,dtype='uint16').reshape([nchannels,nlines,samples_per_pixel,-1]).sum(axis = 2)#.reshape([nchannels,-1,nlines]).astype('uint16')
frame.shape

In [ ]:
%matplotlib notebook
import pylab as plt
plt.imshow(frame[0])

In [ ]:
sbox_set_status_message(box.usb)

In [ ]:
box.scan()

In [ ]:
box.set_continuous_resonant(True)


In [ ]:
box.set_linescan_mode('bidi')


In [ ]:
from scanbox.nlw_controller import box_commands
box.cmd_queue.put(box_commands['version'])


In [ ]:
box.abort()